In [1]:
# train_model.py
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# 1. Veriyi Yükle
df = pd.read_csv("Electric_Vehicle_Population_Data.csv")

# 2. Veri Ön İşleme (Sadece Menzil bilgisi olan araçları alalım)
# Menzili 0 olanlar veya eksik olanlar eğitim için yanıltıcı olabilir.
data_clean = df[df['Electric Range'] > 0].copy()

# Kullanılacak Özellikler
# Hedef: Electric Range
# Girdiler: Model Year, Make, Electric Vehicle Type
X = data_clean[['Model Year', 'Make', 'Electric Vehicle Type']]
y = data_clean['Electric Range']

# Eğitim ve Test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Pipeline Kurulumu (Kategorik verileri otomatik dönüştürmek için)
numeric_features = ['Model Year']
categorical_features = ['Make', 'Electric Vehicle Type']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Model Tanımlama (Random Forest)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# 4. Modeli Eğitme
print("Model eğitiliyor, lütfen bekleyin...")
pipeline.fit(X_train, y_train)

# 5. Değerlendirme
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Model Eğitimi Tamamlandı.")
print(f"Mean Squared Error: {mse:.2f}")
print(f"R2 Score: {r2:.4f}")

# 6. Modeli Kaydetme
joblib.dump(pipeline, 'ev_range_model.pkl')
print("Model 'ev_range_model.pkl' olarak kaydedildi.")

Model eğitiliyor, lütfen bekleyin...
Model Eğitimi Tamamlandı.
Mean Squared Error: 141.33
R2 Score: 0.9857
Model 'ev_range_model.pkl' olarak kaydedildi.
